In [1]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import Model, load_model

C:\Users\Lenovo\Anaconda3\envs\SelfDrivingProject\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def process_image(image):
    #gray = np.expand_dims(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), axis=2)
    #gray = (gray/255 - 0.5).astype(np.float32)
    reshapeImage = np.expand_dims(image[380:600,620:], axis=0)
    predImage = model.predict(reshapeImage, batch_size=1)
    results = np.zeros_like(image)
    results[380:540,620:,0] = predImage[0,:160,:,0]
    results[380:540,620:,1] = predImage[0,:160,:,0]
    results[380:540,620:,2] = predImage[0,:160,:,0]
    #get_model_memory_usage(1,model)640
    result = cv2.addWeighted(image, 1, results, 0.7, gamma=0)
    return result

In [3]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'output_images/project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")#.subclip(40,45)
# load the CCN trained model
model = load_model('Car_Model.h5')
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)
HTML("""<video width="960" height="540" controls><source src="{0}"></video>""".format(white_output))

[MoviePy] >>>> Building video output_images/project_video.mp4
[MoviePy] Writing video output_images/project_video.mp4


100%|█████████████████████████████████████▉| 1260/1261 [01:30<00:00, 13.91it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video.mp4 

Wall time: 1min 31s
